In [1]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
import time
import sys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import pandas as pd

In [2]:
exeDriver = "chromedriver.exe"
driver = webdriver.Chrome(exeDriver)
print("Iniciando Scraper")

Iniciando Scraper


### Ingresar a busqueda de objetos de aprendizaje en Ingles

In [3]:
url_ = "https://www.merlot.org/merlot/"
driver.get(url_)

In [4]:
search_advanced = driver.find_element_by_class_name("hp-adv-search")
search_advanced.click()

In [5]:
#types_selections = driver.find_elements_by_class_name("adv-search-col")
#material_type_learning_object = types_selections[0].find_elements_by_class_name("custom-control")
#material_type_learning_object[10].click()

In [6]:
language_selection = Select(driver.find_element_by_id("language"))
language_selection.select_by_value("eng")

In [7]:
search = driver.find_element_by_class_name("mr-3")
search.click()

### Obteniendo metadatos articulos

In [8]:
columns = ['title', 'description', 'keywords', 'disciplines', 'material_type', 'date_added_merlot', 'date_modified_merlot',
           'authors', 'submitter', 'primary_audience', 'technical_format', 'mobile_compatibility', 'language',
           'cost_involved', 'source_code_available', 'accessibility_information_available', 'creative_commons', 'url']
df = pd.DataFrame(columns = columns)

In [9]:
def get_metadata_page():
    # Seccion Material Detail
    material_detail = driver.find_elements_by_class_name('row')[2]
    material_detail_txt = material_detail.text
    material_detail_list = material_detail_txt.split('\n')

    url = driver.current_url
    title = material_detail_list[0]
    description = material_detail_list[1]
    keywords = ''
    disciplines = ''
    for i in range(len(material_detail_list)):
        if (material_detail_list[i] == 'Keywords:'):
            keywords = material_detail_list[i+1]
        if (material_detail_list[i] == 'Disciplines:'):
            disciplines = material_detail_list[i+1]

    # Seccion More about this material
    more_about_material = driver.find_elements_by_class_name('row')[7]
    more_about_material_txt = more_about_material.text
    more_about_material_list = more_about_material_txt.split('\n')
    more_about_material_list

    material_type = ''
    date_added_merlot = ''
    date_modified_merlot = '' 
    authors = ''
    submitter = ''
    primary_audience = ''
    technical_format = ''
    mobile_compatibility = ''
    language = ''
    cost_involved = ''
    source_code_available = ''
    accessibility_information_available = ''
    creative_commons = ''
    for i in range(len(more_about_material_list)):
        if (more_about_material_list[i] == 'Material Type:'):
            material_type = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Date Added to MERLOT:'):
            date_added_merlot = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Date Modified in MERLOT:'):
            date_modified_merlot = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Author:'):
            for j in range(i+1, len(more_about_material_list)):
                if (more_about_material_list[j] == 'Submitter:'):
                    break
                else:
                    authors += more_about_material_list[j] + ', '
        if (more_about_material_list[i] == 'Submitter:'):
            submitter = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Primary Audience:'):
            primary_audience = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Technical Format:'):
            technical_format = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Mobile Compatibility:'):
            mobile_compatibility = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Language:'):
            language = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Cost Involved:'):
            cost_involved = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Source Code Available:'):
            source_code_available = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Accessibility Information Available:'):
            accessibility_information_available = more_about_material_list[i+1]
        if (more_about_material_list[i] == 'Creative Commons:'):
            creative_commons = more_about_material_list[-1]
            
    diccionario = {'title': title, 'description': description, 'keywords': keywords, 'disciplines': disciplines, 
                   'material_type': material_type, 'date_added_merlot': date_added_merlot, 
                   'date_modified_merlot': date_modified_merlot, 'authors': authors, 'submitter': submitter, 
                   'primary_audience': primary_audience, 'technical_format': technical_format, 
                   'mobile_compatibility': mobile_compatibility, 'language': language, 
                   'cost_involved': cost_involved, 'source_code_available': source_code_available, 
                   'accessibility_information_available': accessibility_information_available, 
                   'creative_commons': creative_commons, 'url': url}
    return diccionario

### Iterar sobre todos los objetos de aprendizaje

In [12]:
for i in range(418):
    learning_objects_list = driver.find_elements_by_class_name("card.merlot-material-item")
    for learning_object in learning_objects_list:
        try:
            actions = ActionChains(driver)
            actions.key_down(Keys.CONTROL).click(learning_object.find_element_by_tag_name("a")).key_up(Keys.CONTROL).perform()
            driver.switch_to.window(driver.window_handles[-1])
            diccionario = get_metadata_page()   
            df = df.append(diccionario, ignore_index=True)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            print("fallo en una pagina")
            pass
    try:
        pagination = driver.find_element_by_class_name("col.merlot-pagination").find_elements_by_tag_name("li")
        next_page = pagination[-2]
        next_page.click()
    except:
        print("Fallo al pasar de pagina")
        pass

fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
fallo en una pagina
Fallo al pasar de pagina


In [13]:
df

,title,description,keywords,disciplines,material_type,date_added_merlot,date_modified_merlot,authors,submitter,primary_audience,technical_format,mobile_compatibility,language,cost_involved,source_code_available,accessibility_information_available,creative_commons,url
0,Authentic Assessment Toolbox,The Authentic Assessment Toolbox site is a tut...,,Academic Support Services / Faculty Developme...,Tutorial,"January 6, 2003","October 16, 2019","Jon Mueller, North Central College, IL Email...",Cris Guenter,"College General Ed, Graduate School, Professional",Website,Not specified at this time,English,No,No,No,This work is licensed under a Attribution-NonC...,https://www.merlot.org/merlot/viewMaterial.htm...
1,DNA from the Beginning,DNA from the Beginning is an animated tutorial...,"genetics, genes, DNA, DNA models, History abou...",Science and Technology / Agriculture and Envi...,Simulation,"April 11, 2000","October 11, 2019","Cold Spring Harbor Laboratory,",Jeff Bell,College General Ed,Website,Not specified at this time,English,No,No,No,Unknown,https://www.merlot.org/merlot/viewMaterial.htm...
2,Mathematical Visualization Toolkit,This site consists of a collection of plotting...,"implicit, polar, vector field, Taylor, spheric...",Mathematics and Statistics / Mathematics / C...,Simulation,"July 17, 2001","October 18, 2019","University of Colorado at Boulder, Department ...",Kurt Cogswell,College General Ed,"Website, Java Applet",Not specified at this time,English,No,No,No,No,https://www.merlot.org/merlot/viewMaterial.htm...
3,Neuroscience for Kids,The entry point to an extensive site concernin...,"psychology, neuroscience, kids",Education / TeacherEd / Teaching Methods / ...,Collection,"November 30, 2000","October 15, 2019","Eric Chudler, Dept. Anesthesiology; University...",Frank Leeming,College General Ed,Other,Not specified at this time,English,No,Unknown,No,No,https://www.merlot.org/merlot/viewMaterial.htm...
4,Assessing Blood Pressure,ADOBE FLASH REQUIRED. This learning module pr...,,Workforce Development / Technical Allied Heal...,Tutorial,"April 21, 2004","October 15, 2019","Andrew Winterstein, University of Wisconsin - ...",Andrew Winterstein,College General Ed,Website,Not specified at this time,English,No,No,No,No,https://www.merlot.org/merlot/viewMaterial.htm...
5,Crisis at Fort Sumter,"""Crisis at Fort Sumter"" is an interactive hist...",,Humanities / History / Area Studies / Ameri...,Simulation,"March 26, 2003","September 10, 2019","Richard B. Latner, Tulane University,",Cynthia Poe,"High School, College General Ed",,Not specified at this time,English,No,No,No,No,https://www.merlot.org/merlot/viewMaterial.htm...
6,FAST (Free Assessment Summary Tool),This is a powerful--and free!--program that fa...,,Academic Support Services / Faculty Developme...,Quiz/Test,"October 31, 2002","October 2, 2019","Bruce D. Ravelli, Mount Royal College, Canada ...",Thomas Nolan,College General Ed,,Not specified at this time,English,No,No,No,No,https://www.merlot.org/merlot/viewMaterial.htm...
7,Guide to Grammar and Writing,"Full of great tutorials, this website provides...",,Academic Support Services / Library and Infor...,Tutorial,"November 18, 2003","October 10, 2019","Charles Darling, Capitol Community College,",Joseph Milford,"High School, College General Ed, Graduate Scho...",Website,Not specified at this time,English,No,No,No,No,https://www.merlot.org/merlot/viewMaterial.htm...
8,Ideal Gas Law Simulation,An effective tool to help students learn about...,,Science and Technology / Chemistry,Simulation,"February 19, 2001","October 19, 2019","John Gelder, Oklahoma State University, Depart...",Tom Greenbowe,College General Ed,"Java Applet, Website",Not specified at this time,English,No,No,No,Unknown,https://www.merlot.org/merlot/viewMaterial.htm...
9,LangMedia Foreign Language Media Archive,This site presents information on culture and ...,,Humanities / World Languages / Multilingual ...,Simulation,"January 10, 2002","September 20, 2019","

In [14]:
df.to_excel("dataEXCEL.xlsx", "dataEXCEL")